In [1]:
reload_lamb()

In [2]:
import contextlib

class MaybeElem:
    def __repr__(self):
        return 'Maybe'

    def __eq__(self, other):
        return isinstance(types.demeta(other), MaybeElem)

    def __hash__(self):
        return hash("MaybeElem")

class MaybeSet(types.ComplexDomainSet):
    def __init__(self, typ):
        finite = isinstance(typ.arg.domain, types.DomainSet) and typ.arg.domain.finite
        super().__init__("M", typ, finite=finite)

    def check(self,x):
        # bool is implemented as a subclass of int, and therefore 0 and 1 are
        # equivalent (both hashing, and via ==) to False and True. Checking
        # for set membership in {False,True} includes 0/1 in the domain.
        # This perhaps could be useful if we have a way of normalizing the
        # term names, but for now, instead of using the superclass `in`
        # check, exclude 0/1 from the domain by checking class directly.
        return (isinstance(types.demeta(x), MaybeElem)
                or isinstance(self.type.arg.domain, types.DomainSet) and self.type.arg.domain.check(x))

    def __len__(self):
        if not self.finite:
            raise ValueError("Non-finite `MaybeDomainSet`s do not have a length.")
        return len(self.type.arg.domain) + 1

    def __iter__(self):
        if not self.finite:
            raise ValueError("Can't iterate over non-finite `DisjunctiveDomainSet`s.")
        if MaybeElem() not in self.type.arg.domain:
            yield MaybeElem()
        yield from iter(self.type.arg.domain)
    
    def enumerable(self):
        # only the finite case is supported right now...
        return self.finite

    def normalize(self, x):
        if x in self.type.arg.domain:
            return self.type.arg.domain.normalize(x)
        else:
            return MaybeElem()

    def element_repr(self, x, rich=False, **kwargs):
        if isinstance(types.demeta(x), MaybeElem):
            return "Maybe"
        else:
            return self.type.arg.domain.element_repr(x, rich=rich)

    def __repr__(self):
        return f"MaybeSet({self.type})"

    @classmethod
    def element_to_type(cls, x, ctrl=None):
        raise lamb.parsing.ParseError("`Maybe` terms require a complex type")


class MaybeType(types.TypeConstructor):
    domain_class = MaybeSet
    def __init__(self, c):
        self.arg = c
        types.TypeConstructor.__init__(self)

    def __len__(self):
        return 1

    def __getitem__(self, i):
        return (self.arg,)[i]

    def __iter__(self):
        return iter((self.arg,))
        
    def __repr__(self):
        return f"M{repr(self.arg)}"

    def functional(self):
        return self.arg.functional()

    @property
    def left(self):
        return self.arg.left

    @property
    def right(self):
        return self.arg.right

    def _type_eq(self, other):
        if isinstance(other, MaybeType):
            return self.arg._type_eq(other.arg)
        else:
            return False

    def _type_hash(self):
        return (hash("M") ^ self.arg._type_hash())

    def copy_local(self, arg):
        return MaybeType(arg)

    @classmethod
    def parse(cls, s, i, parse_control_fun):
        next = lamb.parsing.consume_char(s, i, "M")
        if next is None:
            return (None, i)
        else:
            i = next
            (arg, i) = parse_control_fun(s, i)
            return (MaybeType(arg), i)

    def latex_str(self):
        return utils.ensuremath(f"M{self.arg.latex_str()}")

class MaybeExpr(meta.TypedExpr):
    def __init__(self, arg, type_check=True):
        super().__init__("MaybeWrap", arg)
        self.type = MaybeType(arg.type)

    def simplify(self, **sopts):
        if self[0].meta():
            return meta.MetaTerm(self[0].op, typ=self.type)
        else:
            return self
    
    def __repr__(self):
        return f"MaybeWrap({repr(self[0])})"

    def latex_str(self, suppress_parens=False, **kwargs):
        body_str = self[0].latex_str(suppress_parens=True, **kwargs)
        return utils.ensuremath(f"\\textsf{{MaybeWrap}}({body_str})")

    def try_adjust_type_local(self, new_type, derivation_reason, env):
        arg = self[0].try_adjust_type(new_type.arg, derivation_reason=derivation_reason)
        if arg is None:
            return None
        return self.copy_local(arg)

class MaybeTerm(meta.TypedTerm):
    def __init__(self, type_check=True):
        super().__init__("Maybe", typ=MaybeType(types.VariableType.any()))

    # def meta(self):
    #     return True

    def should_show_type(self, assignment=None):
        return True
    
    def simplify(self, **sopts):
        if not self.type.polymorphic():
            return meta.MetaTerm(MaybeElem(), typ=self.type)
        return self

class UnmaybeExpr(meta.TypedExpr):
    def __init__(self, arg, type_check=True):
        if not isinstance(arg.type, MaybeType):
            raise types.TypeMismatch(arg, arg.type, "`Unmaybe` requires a Maybe typed argument")
        super().__init__("Unmaybe", arg)
        self.type = arg.type.arg

    def simplify(self, **sopts):
        if isinstance(self[0], MaybeExpr):
            return self[0][0]
        elif self[0].meta():
            if isinstance(self[0].op, MaybeElem):
                raise meta.meta.DomainError(self[0], self.type.domain, "`Unmaybe(Maybe)` is undefined")
            return meta.MetaTerm(self[0].op, typ=self.type)
        else:
            return self
    
    def __repr__(self):
        return f"Unmaybe({repr(self[0])})"

    def latex_str(self, suppress_parens=False, **kwargs):
        body_str = self[0].latex_str(suppress_parens=True, **kwargs)
        return utils.ensuremath(f"\\textsf{{Unmaybe}}({body_str})")

    def try_adjust_type_local(self, new_type, derivation_reason, env):
        arg = self[0].try_adjust_type(MaybeType(new_type), derivation_reason=derivation_reason)
        if arg is None:
            return None
        return self.copy_local(arg)


class Case(meta.TypedExpr):
    def __init__(self, cond, if_case, else_case, /, type_check=True):
        if cond.type != types.type_t:
            raise types.TypeMismatch(arg, arg.type, "`Cases` requires a type t condition")
        self.type = meta.core.ts_unify_with(if_case, else_case,
                                error="`Case` requires compatible result types")
        if_case = self.ensure_typed_expr(if_case, self.type)
        else_case = self.ensure_typed_expr(else_case, self.type)
        super().__init__("Case", cond, if_case, else_case)

    def simplify(self, **sopts):
        if self[0] == True:
            return self[1]
        elif self[0] == False:
            return self[2]
        else:
            return self

    @classmethod
    def from_tuple(self, tuple):
        return Case(*tuple)
    
    def __repr__(self):
        return f"Case({repr(self[0])}, {repr(self[1])}, {repr(self[2])})"

    def latex_str(self, suppress_parens=False, **kwargs):
        inner = ", ".join([a.latex_str(**kwargs) for a in self.args])
        # inner = f"({inner})"
        return utils.ensuremath(f"\\textsf{{Case}}({inner})")

    def try_adjust_type_local(self, new_type, derivation_reason, env):
        if_case = self[1].try_adjust_type(new_type, derivation_reason=derivation_reason)
        if if_case is None:
            return None
        else_case = self[2].try_adjust_type(new_type, derivation_reason=derivation_reason)
        if else_case is None:
            return None
        return self.copy_local(self[0], if_case, else_case)

    def latex_str(self, **kwargs):
        cases = []
        c = self
        while isinstance(c, Case):
            cases.append((c[1], c[0]))
            c = c[2]
        cases = [f"{case[0].latex_str(**kwargs)} & \\text{{if }}{case[1].latex_str(**kwargs)} \\\\" for case in cases]
        return utils.ensuremath(
            f"\\begin{{cases}}\n{'\n'.join(cases)}"
            f"{c.latex_str(**kwargs)} & \\text{{otherwise}} \\end{{cases}}")
        # return utils.ensuremath(
        #     f"\\begin{{cases}}{self[1].latex_str(**kwargs)} & \\text{{if }}{self[0].latex_str(**kwargs)} \\\\"
        #     f"{self[2].latex_str(**kwargs)} & \\text{{otherwise}} \\end{{cases}}")


Maybe = MaybeElem()

meta.core.reset_type_system()
# meta.reset()
# system = lang.td_system.copy()
meta.get_type_system().add_nonatomic(MaybeType)
# type_m = types.BasicType('m', MaybeSet())
# system.add_basic_type(type_m)
# lang.set_system(system)

lang.get_system().lexicon['Maybe'] = MaybeTerm()
# meta.TypedExpr.add_local('Maybe', MaybeTerm())
# meta.TypedExpr.add_local('Undef', lambda x: x.equivalent_to(MaybeTerm()))
meta.TypedExpr.add_local('MaybeWrap', MaybeExpr)
meta.TypedExpr.add_local('Unmaybe', UnmaybeExpr)
meta.TypedExpr.add_local('Case', Case.from_tuple)



In [3]:
# meta.core.reset_type_system()
meta.get_type_system().type_ranking

{lamb.types.BasicType: 0,
 lamb.types.FunType: 0,
 lamb.types.SetType: 0,
 lamb.types.TupleType: 0,
 lamb.types.UnknownType: 10,
 lamb.types.VariableType: 10,
 lamb.types.Forall: 20,
 lamb.types.DisjunctiveType: 1,
 __main__.MaybeType: 0}

In [4]:
Case(te("p_t"), te('_c1'), te('_c3'))

Case(p_t, _c1, _c3)

In [5]:
Case(te("p_t"), te('_c1'), Case(te('p1_t'), te('_c2'), te('_c3')))

Case(p_t, _c1, Case(p1_t, _c2, _c3))

In [6]:

%te simplify Case(True, _c1, _c2)

_c1

In [7]:
MaybeExpr(te("x_e"))

MaybeWrap(x_e)

In [8]:
UnmaybeExpr(MaybeExpr(te("x_e")))

Unmaybe(MaybeWrap(x_e))

In [9]:
meta.MetaTerm(Maybe, typ=tp("Mt"))

Maybe

In [10]:
with lamb.errors():
    meta.MetaTerm(Maybe)

<span style="color:red">**ParseError**</span>: `Maybe` terms require a complex type

In [11]:
meta.MetaTerm(Maybe, typ=tp("MMt"))

Maybe

In [12]:
meta.MetaTerm(Maybe, typ=tp("MMt")) == meta.MetaTerm(Maybe, typ=tp("Mt"))

True

In [13]:
%te Maybe

Maybe_t

In [14]:
s = te("Set x_Mt : True").eliminate()
s

{False, True, Maybe}

In [15]:
s = te("Set x_MMt : True").eliminate()
s

{False, True, Maybe}

In [16]:
lamb.parsing.errors_raise=False

In [17]:
%%lamb
# Undef = L x_X : x <=> Maybe
unit = L x_X : MaybeWrap(x_X)
bind = L m_MX : L k_<X,MY> : Case(m <=> Maybe, Maybe, k(Unmaybe(m)))


${unit}_{\left\langle{}X,MX\right\rangle{}}\:=\:\lambda{} x_{X} \: . \: \textsf{MaybeWrap}({x})$<br />
${bind}_{\left\langle{}MX,\left\langle{}\left\langle{}X,MY\right\rangle{},MY\right\rangle{}\right\rangle{}}\:=\:\lambda{} m_{MX} \: . \: \lambda{} k_{\left\langle{}X,MY\right\rangle{}} \: . \: \begin{cases}
{Maybe}_{MY} & \text{if }{Maybe}_{MX} = {m} \\{k}(\textsf{Unmaybe}({m})) & \text{otherwise} \end{cases}$

In [18]:
%lamb liftedapply = L f_M<X,Y> : L x_MX : bind(f)(L a_<X,Y> : bind(x)(L b_X : unit(a(b))))


${liftedapply}_{\left\langle{}M\left\langle{}X',X\right\rangle{},\left\langle{}MX',MX\right\rangle{}\right\rangle{}}\:=\:\lambda{} f_{M\left\langle{}X',X\right\rangle{}} \: . \: \lambda{} x_{MX'} \: . \: \begin{cases}
{Maybe}_{MX} & \text{if }{Maybe}_{M\left\langle{}X',X\right\rangle{}} = {f} \\
{Maybe}_{MX} & \text{if }{Maybe}_{MX'} = {x} \\\textsf{MaybeWrap}(\textsf{Unmaybe}({f})(\textsf{Unmaybe}({x}))) & \text{otherwise} \end{cases}$

In [19]:
%lamb lapply = (L m_M∀X : L f_M<X,Y> : L x_MX : Case(x <=> Maybe, Maybe, Case(f <=> Maybe, Maybe, MaybeWrap((Unmaybe(f))(Unmaybe(x))))))


${lapply}_{\left\langle{}M\forall{}X,\left\langle{}M\left\langle{}X,Y\right\rangle{},\left\langle{}MX,MY\right\rangle{}\right\rangle{}\right\rangle{}}\:=\:\lambda{} m_{M\forall{}X} \: . \: \lambda{} f_{M\left\langle{}X,Y\right\rangle{}} \: . \: \lambda{} x_{MX} \: . \: \begin{cases}
{Maybe}_{MY} & \text{if }{Maybe}_{MX} = {x} \\
{Maybe}_{MY} & \text{if }{Maybe}_{M\left\langle{}X,Y\right\rangle{}} = {f} \\\textsf{MaybeWrap}(\textsf{Unmaybe}({f})(\textsf{Unmaybe}({x}))) & \text{otherwise} \end{cases}$

In [20]:
bind(unit(te(True))).simplify_all(reduce=True)

(λ k_<t,MY>: Case((Maybe_Mt <=> True), Maybe_MY, k_<t,MY>(True)))

In [21]:
meta.core.let_wrapper(bind(MaybeTerm()).simplify_all(reduce=True))

(λ k_<X,MX1>: Maybe_MX1)

In [22]:
system = lang.td_system.copy()
lang.set_system(system)
system.add_typeshift(unit, "UnitShift")
system.add_unary_rule(unit, "Unit")

system.add_binary_rule(bind, "Bind")
# system.add_binary_rule(lapply, "LiftedFA")
# do a bit of cleanup, we don't need PM here.
system.remove_rule('PM')
system.typeshift = True
system

Composition system: Type-driven composition (copy)

In [23]:
%te globals L f_<e,t> : Case((ExistsExact x_e : f(x) & Have(T_e, x)), MaybeWrap(Iota x_e : f(x) & Have(T_e, x)), Maybe)

INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'
INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'


(λ f_<e,t>: Case((ExistsExact x_e: (f_<e,t>(x_e) & Have_<(e,e),t>(T_e, x_e))), MaybeWrap((ι x_e: (f_<e,t>(x_e) & Have_<(e,e),t>(T_e, x_e)))), Maybe_Me))

In [24]:
%te globals L p_Mt : L q_Mt : Case(p <=> Maybe, Maybe, Case(p <=> MaybeWrap(False), MaybeWrap(True), q))
# %te globals L p_Mt : L q_Mt : Case(p <=> Maybe, Maybe, q)

(λ p_Mt: (λ q_Mt: Case((p_Mt <=> Maybe_Mt), Maybe_Mt, Case((p_Mt <=> MaybeWrap(False)), MaybeWrap(True), q_Mt))))

In [25]:
%%lamb
||his|| = L f_<e,t> : Case((ExistsExact x_e : f(x) & Have(T_e, x)), MaybeWrap(Iota x_e : f(x) & Have(T_e, x)), Maybe)
||suit|| = L x_e : Suit(x)
||bring|| = L y_e : L x_e : Bring(x,y)
||theo|| = Theo_e
||has_bro|| = L x_e : Bro(x)
||if_|| = L p_Mt : L q_Mt : Case(p <=> Maybe, Maybe, Case(p <=> MaybeWrap(False), MaybeWrap(True), q))

INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'
INFO (core): Coerced guessed type for 'Have_t' into <(e,e),t>, to match argument '(T_e, x_e)'
INFO (core): Coerced guessed type for 'Suit_t' into <e,t>, to match argument 'x_e'
INFO (core): Coerced guessed type for 'Bring_t' into <(e,e),t>, to match argument '(x_e, y_e)'
INFO (core): Coerced guessed type for 'Bro_t' into <e,t>, to match argument 'x_e'


$[\![\text{\textbf{his}}]\!]^{}_{\left\langle{}\left\langle{}e,t\right\rangle{},Me\right\rangle{}} \:=\: \lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {f}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {f}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases}$<br />
$[\![\text{\textbf{suit}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Suit}({x})$<br />
$[\![\text{\textbf{bring}}]\!]^{}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: \lambda{} y_{e} \: . \: \lambda{} x_{e} \: . \: {Bring}({x}, {y})$<br />
$[\![\text{\textbf{theo}}]\!]^{}_{e} \:=\: {Theo}_{e}$<br />
$[\![\text{\textbf{has\_bro}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: \lambda{} x_{e} \: . \: {Bro}({x})$<br />
$[\![\text{\textbf{if\_}}]\!]^{}_{\left\langle{}Mt,\left\langle{}Mt,Mt\right\rangle{}\right\rangle{}} \:=\: \lambda{} p_{Mt} \: . \: \lambda{} q_{Mt} \: . \: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{Mt} = {p} \\
{\textsf{True}}_{Mt} & \text{if }{p} = {\textsf{False}}_{Mt} \\{q} & \text{otherwise} \end{cases}$

In [26]:
bring

⟦bring⟧ = (λ y_e: (λ x_e: Bring_<(e,e),t>(x_e, y_e)))

In [27]:
his * suit

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[his suit]}}]\!]^{}_{Me} \:=\: \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases}$

In [28]:
((lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None)) * (theo * None)).tree()

1 composition path:<br /><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:block;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div style="display:table;table-layout:auto;" align="center"><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>$[\![\text{\textbf{1}}]\!]^{}$</span></div></div></div><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>[idx: 1]</span></div></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:block;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div style="display:table;table-layout:auto;" align="center"><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>$[\![\text{\textbf{theo}}]\!]^{}_{e}$</span></div></div></div><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;"><span>${Theo}_{e}$</span></div></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div align="center" style="display:block;padding-left:5px;padding-right:5px;padding-top:5px;padding-bottom:5px;"><div style="display:table;table-layout:auto;" align="center"><div style="display:table-row;"><div style="display:table-cell;padding-right:2px; padding-left:2px;" align="center"><div style="display:inline-block;

In [29]:
bind

(λ m_MX: (λ k_<X,MY>: Case((Maybe_MX <=> m_MX), Maybe_MY, k_<X,MY>(Unmaybe(m_MX)))))

In [30]:
(his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[his suit] [1 [[theo [bring t1]]]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases}))) & \text{otherwise} \end{cases}$

In [31]:
if_ * ((theo * has_bro) * None)

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[if\_ [[theo has\_bro]]]}}]\!]^{}_{\left\langle{}Mt,Mt\right\rangle{}} \:=\: \lambda{} q_{Mt} \: . \: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{\textsf{True}}_{Mt} & \text{if }{\textsf{False}}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\{q} & \text{otherwise} \end{cases}$

In [32]:
if_ * ((theo * has_bro) * None) * ((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None)))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[if\_ [[theo has\_bro]]] [[his suit] [1 [[theo [bring t1]]]]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{\textsf{True}}_{Mt} & \text{if }{\textsf{False}}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{Maybe}_{Mt} & \text{if }{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases}))) & \text{otherwise} \end{cases}$

Observation: we can get the local reading by scoping out at a lower type??

In [33]:
lang.Binder(2) * ((if_ * ((theo * has_bro) * None)) * (lang.Trace(2, typ=tp('Mt'))))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[2 [[if\_ [[theo has\_bro]]] t2]]}}]\!]^{}_{\left\langle{}Mt,Mt\right\rangle{}} \:=\: \lambda{} x_{Mt} \: . \: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{\textsf{True}}_{Mt} & \text{if }{\textsf{False}}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\{x} & \text{otherwise} \end{cases}$

In [34]:
((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None))) * (lang.Binder(2) * ((if_ * ((theo * has_bro) * None)) * (lang.Trace(2, typ=tp('Mt')))))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[[his suit] [1 [[theo [bring t1]]]]] [2 [[if\_ [[theo has\_bro]]] t2]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{\textsf{True}}_{Mt} & \text{if }{\textsf{False}}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{Maybe}_{Mt} & \text{if }{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases}))) & \text{otherwise} \end{cases}$

In [35]:
((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None * None)))

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[his suit] [1 [[[theo [bring t1]]]]]]}}]\!]^{}_{MMt} \:=\: \begin{cases}
{Maybe}_{MMt} & \text{if }{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}(\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases})))) & \text{otherwise} \end{cases}$

In [36]:
g_reading = ((his * suit) * (lang.Binder(1) * ((theo * (bring * lang.Trace(1))) * None * None))) * (lang.Binder(2) * ((if_ * ((theo * has_bro) * None)) * (lang.Trace(2, typ=tp('Mt')))))
g_reading

1 composition path.  Result:<br />
&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\text{\textbf{[[[his suit] [1 [[[theo [bring t1]]]]]] [2 [[if\_ [[theo has\_bro]]] t2]]]}}]\!]^{}_{Mt} \:=\: \begin{cases}
{Maybe}_{Mt} & \text{if }{Maybe}_{MMt} = \begin{cases}
{Maybe}_{MMt} & \text{if }({Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\{Maybe}_{Me} & \text{otherwise} \end{cases}) \\\textsf{MaybeWrap}(\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases})))) & \text{otherwise} \end{cases} \\
{Maybe}_{Mt} & \text{if }{Maybe}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\
{\textsf{True}}_{Mt} & \text{if }{\textsf{False}}_{Mt} = \textsf{MaybeWrap}({Bro}({Theo}_{e})) \\\textsf{Unmaybe}(\begin{cases}
{Maybe}_{MMt} & \text{if }{Maybe}_{Me} = \begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }(\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) \\{Maybe}_{Me} & \text{otherwise} \end{cases} \\\textsf{MaybeWrap}(\textsf{MaybeWrap}({Bring}({Theo}_{e}, \textsf{Unmaybe}(\begin{cases}
\textsf{MaybeWrap}(\iota{} x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x})) & \text{if }\exists{}! x_{e} \: . \: {Have}({T}_{e}, {x}) \wedge{} {Suit}({x}) \\{Maybe}_{Me} & \text{otherwise} \end{cases})))) & \text{otherwise} \end{cases}) & \text{otherwise} \end{cases}$

In [ ]:
g_reading[0].content